---
# Section 2.7: Backward Error Analysis of Gaussian Elimination
---

## New notation

Let $C \in \mathbb{R}^{m \times n}$. Then $|C|$ is the $m \times n$ matrix with entries $|c_{ij}|$ and is called the **absolute value** of $C$.

If $C, F \in \mathbb{R}^{m \times n}$, then $C \leq F$ means $c_{ij} \leq f_{ij}$, for all $ij$.

We let $u$ be the **unit-roundoff**, and we use $O(u^2)$ to denotes terms of order $u^2$.


In [ ]:
A = randn(3,4)

In [ ]:
abs.(A)

In [ ]:
B = randn(3, 4)

In [ ]:
A .<= B

In [ ]:
all(A .<= B)

In [ ]:
A = rand(3,3)

In [ ]:
B = 2*A

In [ ]:
A .<= B

In [ ]:
all(A .<= B)

---

>### Theorem: (Backward Error of LU)
>
> Let $\hat{L}$ and $\hat{U}$ be the $LU$-factors of $A$ computed by Gaussian elimination in floating-point arithmetic without row-exchanges. 
>
> Then
>
> $$A + E = \hat{L} \hat{U}$$
>
> where
>
> $$\lvert E \rvert \leq 2nu \lvert \hat{L} \rvert \lvert \hat{U} \rvert + O(u^2)$$
>
> and
>
> $$\lVert E \rVert_\infty \leq 2nu \lVert \hat{L} \rVert_\infty \lVert \hat{U} \rVert_\infty + O(u^2).$$


---

>### Theorem: (Solving $Ax=b$ via Gaussian Elimination)
>
> Let $\hat{L}$ and $\hat{U}$ be the $LU$-factors of $A$ computed by Gaussian elimination in floating-point arithmetic without row-exchanges. 
>
> Let $\hat{x}$ be the solution obtained from solving $Ax = b$ in floating-point arithmetic using forward substitution with $\hat{L}$ and backward substitution with $\hat{U}$.
>
> Then
>
> $$(A + \delta A)\hat{x} = b$$
>
> where
>
> $$\lvert \delta A \rvert \leq 6nu \lvert \hat{L} \rvert \lvert \hat{U} \rvert + O(u^2)$$
>
> and
>
> $$\lVert \delta A \rVert_\infty \leq 6nu \lVert \hat{L} \rVert_\infty \lVert \hat{U} \rVert_\infty + O(u^2).$$


---

## Backward stability

- If $\lVert \hat{L} \rVert_\infty \lVert \hat{U} \rVert_\infty$ is small compared to $\lVert A \rVert_\infty$, then computation is **backwards stable**.

- Using **partial pivoting** we have

    $$\lVert \hat{L} \rVert_\infty \leq n$$

    In this case, if $\lVert \hat{U} \rVert_\infty$ is small compared to $\lVert A \rVert_\infty$, then computation is **backwards stable**.
    

---

## Exercise:

Compute $U$ of the $LU$-decomposition (using **partial pivoting**) of the matrix $A$.

$$
A = 
\begin{bmatrix}
1 & & & 1 \\
-1 & 1 & & 1 \\
-1 & -1 & 1 & 1 \\
-1 & -1 & -
1 & 1 \\
\end{bmatrix}
$$

---

- Therefore, it is possible that

    $$\frac{\lVert \hat{U} \rVert_\infty}{\lVert A \rVert_\infty} \approx 2^{n-1}.$$
    
    So Gaussian elimination with partial pivoting is *not* guaranteed to be backwards stable.

---

## Complete pivoting

- With **complete pivoting**, we will interchange both rows and columns at each step to bring the entry having the largest magnitude to the current pivot location.

---

## Exercise:

Compute $U$ of the $LU$-decomposition (using **complete pivoting**) of the matrix $A$.

$$
A = 
\begin{bmatrix}
1 & & & 1 \\
-1 & 1 & & 1 \\
-1 & -1 & 1 & 1 \\
-1 & -1 & -
1 & 1 \\
\end{bmatrix}
$$

---

- Gaussian elimination with **complete pivoting** is considered to be backward stable since the worst known case is 

    $$\frac{\lVert \hat{U} \rVert_\infty}{\lVert A \rVert_\infty} = O(n).$$

- **Complete pivoting** is significantly more expensive than partial pivoting, so partial pivoting is preferred in practice since we typically have

    $$\frac{\lVert \hat{U} \rVert_\infty}{\lVert A \rVert_\infty} \approx \sqrt{n}$$
    
    when using partial pivoting.

---

## Residual test and iterative refinement

For the reasons mentioned above, the main method for solving $Ax = b$ is Gaussian elimination with partial pivoting.

We can easily check the computation of $\hat{x}$ was backward stable by computing the residual:

$$
\hat{r} = b - A \hat{x}.
$$

If $\frac{\| \hat{r} \|}{\|b\|}$ is not tiny, we can often improve the solution using **iterative refinement**:

1. Solve $A z = \hat{r}$ using the $LU$-decomposition of $A$ that has already been computed; obtain the numerical solution $\hat{z}$.

2. Update $\hat{x} \gets \hat{x} + \hat{z}$.

3. If $\frac{\| \hat{r} \|}{\|b\|}$ is still large, repeat.

If $A \hat{z} = \hat{r}$, then

$$
A\left(\hat{x} + \hat{z}\right) = A\hat{x} + A\hat{z} = A\hat{x} + \hat{r} = b,
$$

so $\hat{x} + \hat{z}$ is the exact solution of $Ax = b$.

---

## Example:

In [ ]:
using LinearAlgebra

In [ ]:
# Random problem with exact solution x
n = 1000

A = randn(n, n)
x = randn(n)
b = A*x

# Check condition number of A
cond(A)

In [ ]:
# Compute the LU-decomposition of A
F = lu(A)

xhat = F\b

rhat = b - A*xhat

@show norm(rhat)/norm(b)
@show norm(x - xhat)/norm(x);

In [ ]:
# Iterative refinement
zhat = F\rhat
xhat += zhat

# Check new relative residual error
rhat = b - A*xhat

@show norm(rhat)/norm(b)
@show norm(x - xhat)/norm(x);

In [ ]:
# Iterative refinement
zhat = F\rhat
xhat += zhat

# Check new relative residual error
rhat = b - A*xhat

@show norm(rhat)/norm(b)
@show norm(x - xhat)/norm(x);

In [ ]:
# Iterative refinement
zhat = F\rhat
xhat += zhat

# Check new relative residual error
rhat = b - A*xhat

@show norm(rhat)/norm(b)
@show norm(x - xhat)/norm(x);

---

## Stability of Cholesky's method

Cholesky's method of decomposing a positive definite matrix $A$ as $A = R^T R$ is guaranteed to be backward stable since

$$
A + E = \hat{R}^T \hat{R}, \qquad \text{where $\|E\|_F \approx Cn\|A\|_F$},
$$

and $C$ is a small constant.

---

## Rule of thumb

If we solve $Ax = b$ numerically using Gaussian elimination with partial pivoting or Cholesky's method (if $A$ is positive definite), and the entries of $A$ and $b$ are accurate to $s$ decimal places, then $\hat{x}$ will agree with $x$ to $s - t$ decimal places if $\kappa(A) \approx 10^t$.

---

In [ ]:
# Random problem with exact solution x
n = 1000

A = randn(n, n)
x = randn(n)
b = A*x

# Check that A is well-conditioned
cond(A)

In [ ]:
t = log10(ans)

In [ ]:
s = 16

In [ ]:
# Numerically solve Ax = b and compute the residual
xhat = A\b

rhat = b - A*xhat

norm(rhat)/norm(b)

In [ ]:
# Actual relative error
norm(x - xhat)/norm(x)

In [ ]:
s - t

In [ ]:
using Printf
@printf "x[1] = %20.16f\n" x[1]
@printf "x̂[1] = %20.16f\n" xhat[1]

---